## PLX - Phuture Memecoin Index - Methodology Template

## Configuring packages

In [1]:
import pandas as pd
import decouple
import sys
sys.path.append('../')
from methodology import MethodologyBase
from pycoingecko import CoinGeckoAPI
key =  decouple.config("CG_KEY")
cg = CoinGeckoAPI(api_key=key)

## Index Variables

In [2]:
min_mcap =  0.25e8
min_weight = 0.005
max_weight = 0.3
circ_supply_threshold = 0.2
liveness_threshold = 180 # In days
liquidity_consistency = 90 # In days
coingecko_category = ["meme-token"]

# Calculating max slippage based on liquidity threshold
liq = 2.5*1e6
liq_os = liq / 2 # one-side liquidity 
trade_value_tiny = 1e2
trade_value = 1e5
max_slippage = (liq_os + trade_value_tiny) / (liq_os + trade_value) - 1
# round to the nearest .01
max_slippage = round(max_slippage, 2)

# Dataframes to remove
stablecoins = pd.DataFrame(cg.get_coins_markets('usd',category='stablecoins')).set_index('id')
ctokens = pd.DataFrame(cg.get_coins_markets('usd',category='compound-tokens')).set_index('id')
atokens = pd.DataFrame(cg.get_coins_markets('usd',category='aave-tokens')).set_index('id')

methodology_class = MethodologyBase(
                                'arbitrum-one',
                                min_mcap,
                                min_weight,
                                max_weight,
                                65535,
                                circ_supply_threshold,
                                liveness_threshold,
                                liquidity_consistency,
                                max_slippage,
                                100000,
                                coingecko_category)

### Token inclusion criteria

1. **Project and token characteristics**
    1. The project’s token should have been listed on CoinGecko with pricing data spanning at least 6 months prior to the date of inclusion in the index.
    1. The project should have an active community.
    1. No rebasing or deflationary tokens.
    1. Synthetic tokens which derive their value from external price feeds are not permissible. 
    1. The project’s token must not have the ability to pause token transfers.
    1. The project must have a minimum circulating market cap of $50mm.
   <br>
1. **Pricing requirmeents**
    1. Token should have a reliable price feed from one of our authorised providers. 
    <br>
1. **Token Supply requirements**
    1. The project's token must have a circulating supply greater than 20% of the max supply. In cases where a token does not have a max supply, the minting mechanics would need to be assessed.
    <br>
1. **Liquidity Requirements**
    1. The token must be listed on a supported exchange.
    1. The price should experience no more than 12% price impact when executing a $100k trade.
    <br>
1. **Security Requirements**
    1. The project must have been audited by smart contract security professionals with the audit report(s) publicly available. Alternatively, the protocol must have been operating long enough to create a consensus about its safety in the decentralised finance community.
1. **Weighting requirements**
    1. The maximum weight any one token can have is 30%.
    1. All excess weight is proportionally redistributed to all uncapped tokens. After this has been completed, if another token now exceeds the 30% threshold the excess will be redistributed to the remaining uncapped tokens. This process will occur iteratively until there are no tokens that exceed the maximum weight.
    1. Any asset with a weight below 0.5% will be removed from the index.


## Run Methodology

In [3]:

results, liquidity = methodology_class.main(
    df_to_remove=[stablecoins,ctokens,atokens],
    )
liquidity

Excluding based-brett, pricing data available only for 133 < 180 days
Excluding apu-s-club, pricing data available only for 120 < 180 days
Excluding peipeicoin-vip, pricing data available only for 34 < 180 days
Excluding kendu-inu, pricing data available only for 94 < 180 days
Excluding andy-the-wisguy, pricing data available only for 122 < 180 days
Excluding wuffi, pricing data available only for 91 < 180 days
Excluding degen-base, pricing data available only for 178 < 180 days
Excluding maga-hat, pricing data available only for 52 < 180 days
Excluding hoppy-meme, pricing data available only for 81 < 180 days
Excluding why, pricing data available only for 94 < 180 days
Excluding basenji, pricing data available only for 107 < 180 days
Excluding crow-with-knife, pricing data available only for 119 < 180 days
Excluding puff-the-dragon, pricing data available only for 121 < 180 days
Excluding zyncoin-2, pricing data available only for 127 < 180 days
Excluding landwolf-0x67, pricing data a

,arbitrum-one,avalanche,base,binance-smart-chain,ethereum,polygon-pos,optimal chain,best slippage,optimal slippage,best slippage chain
id,,,,,,,,,,
pepe,-0.333333,NaN,NaN,-0.078029,-0.004613,NaN,ethereum,-0.004613,-0.004613,ethereum
floki,NaN,NaN,NaN,-0.999900,-0.007669,NaN,ethereum,-0.007669,-0.007669,ethereum
mog-coin,NaN,NaN,-0.120499,NaN,-0.010199,NaN,ethereum,-0.010199,-0.010199,ethereum
shiba-inu,NaN,NaN,NaN,NaN,-0.015142,NaN,ethereum,-0.015142,-0.015142,ethereum
maga,NaN,NaN,NaN,-0.999900,-0.023441,NaN,ethereum,-0.023441,-0.023441,ethereum
pepecoin-2,NaN,NaN,NaN,NaN,-0.025199,NaN,ethereum,-0.025199,-0.025199,ethereum
turbo,NaN,NaN,NaN,NaN,-0.028105,NaN,ethereum,-0.028105,-0.028105,ethereum
harrypotterobamasonic10in,NaN,NaN,NaN,NaN,-0.043369,NaN,ethereum,-0.043369,-0.043369,ethereum
bone-shibaswap,NaN,NaN,NaN,NaN,-0.044724,NaN,ethereum,-0.044724,-0.044724,ethereum


In [4]:
results

,symbol,market_cap,weight,weight_converted,address,blockchain
id,,,,,,
shiba-inu,SHIB,9707695338,0.3,19660,0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce,ethereum
pepe,PEPE,3902981954,0.3,19661,0x6982508145454ce325ddbe47a25d4ec3d2311933,ethereum
floki,FLOKI,1479759632,0.3,19661,0xcf0c122c6b73ff809c693db761e7baebe62b6a2e,ethereum
bone-shibaswap,BONE,116110750,0.1,6553,0x9813037ee2218799597d83d4a5b6f3b6778218d9,ethereum


In [1]:
methodology_class.slippage_data

NameError: name 'methodology_class' is not defined